In [2]:
# Generate a report of the cluster state
!hdfs dfsadmin -fs hdfs://main:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 8874823680 (8.27 GB)
DFS Remaining: 8874799104 (8.27 GB)
DFS Used: 24576 (24 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.3:9866 (hdfs_practice-hdfs-1.hdfs_practice_default)
Hostname: main
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 16929452032 (15.77 GB)
DFS Remaining: 8874799104 (8.27 GB)
DFS Used%: 0.00%
DFS Remaining%: 34.37%
Configured Cache Capacit

In [11]:
!hdfs dfs -mkdir hdfs://main:9000/data

In [14]:
# As an example, let's load the software license (from notebook container) into hdfs
!head /hadoop-3.3.6/LICENSE.txt


                                 Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,


In [12]:
# What files do we have in hdfs so far?
!hdfs dfs -ls hdfs://main:9000/

Found 1 items
drwxr-xr-x   - root supergroup          0 2023-10-20 01:25 hdfs://main:9000/data


In [15]:
# copy from container to hdfs
!hdfs dfs -cp /hadoop-3.3.6/LICENSE.txt hdfs://main:9000/data

In [17]:
# Now the file shows in hdfs
!hdfs dfs -ls hdfs://main:9000/data

Found 1 items
-rw-r--r--   3 root supergroup      15217 2023-10-20 01:29 hdfs://main:9000/data/LICENSE.txt


In [19]:
# We can look at the contents (cat, head, tail) of the file we just loaded into hdfs
!hdfs dfs -head hdfs://main:9000/data/LICENSE.txt


                                 Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whether by contract or
      otherwise, or (ii) ownership of fifty percent (50%) or more of the
      outstanding shares, or (iii) beneficial own

In [22]:
# how much disk space is being used?
!hdfs dfs -du hdfs://main:9000/data

15217  45651  hdfs://main:9000/data/LICENSE.txt


In [23]:
# we actually get 2 numbers (45651=physical bytes, 15217=logical bytes)
# also note, 45651 is the "expected" physical bytes, but we only have 1 data node, therefore only 1 copy of the file
45651 / 15217

3.0

In [27]:
# fsck = file system checker; helps us figure out if there's an issue with the file (ex. we currently only have 1 replica when we need 3)
# note the "Missing Replicas" in the Replicated Blocks section that indicates this problem; also block is "Under-replicated"
!hdfs fsck hdfs://main:9000/data/LICENSE.txt

Connecting to namenode via http://main:9870/fsck?ugi=root&path=%2Fdata%2FLICENSE.txt
FSCK started by root (auth:SIMPLE) from /172.18.0.2 for path /data/LICENSE.txt at Fri Oct 20 01:38:06 GMT 2023


/data/LICENSE.txt:  Under replicated BP-1309233855-172.18.0.3-1697762626319:blk_1073741825_1001. Target Replicas is 3 but found 1 live replica(s), 0 decommissioned replica(s), 0 decommissioning replica(s).

Status: HEALTHY
 Number of data-nodes:	1
 Number of racks:		1
 Total dirs:			0
 Total symlinks:		0

Replicated Blocks:
 Total size:	15217 B
 Total files:	1
 Total blocks (validated):	1 (avg. block size 15217 B)
 Minimally replicated blocks:	1 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	1 (100.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	3
 Average block replication:	1.0
 Missing blocks:		0
 Corrupt blocks:		0
 Missing replicas:		2 (66.666664 %)
 Blocks queued for replication:	0

Erasure Coded Block Groups:
 Total size:	0 B
 Total files:	0

In [29]:
# let's copy the file in again, but make sure that hdfs doesn't think it's underreplicated this time (i.e. only expects 1 replica)
# hdfs dfs <OPTIONS_TO_JAVA> -cp <src> <dst> 
!hdfs dfs -D dfs.replication=1 -cp /hadoop-3.3.6/LICENSE.txt hdfs://main:9000/data/v2.txt

In [31]:
# use fsck to verify that hdfs thinks this v2 copy is good to go
!hdfs fsck hdfs://main:9000/data/v2.txt

Connecting to namenode via http://main:9870/fsck?ugi=root&path=%2Fdata%2Fv2.txt
FSCK started by root (auth:SIMPLE) from /172.18.0.2 for path /data/v2.txt at Fri Oct 20 01:41:49 GMT 2023


Status: HEALTHY
 Number of data-nodes:	1
 Number of racks:		1
 Total dirs:			0
 Total symlinks:		0

Replicated Blocks:
 Total size:	15217 B
 Total files:	1
 Total blocks (validated):	1 (avg. block size 15217 B)
 Minimally replicated blocks:	1 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	0 (0.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	3
 Average block replication:	1.0
 Missing blocks:		0
 Corrupt blocks:		0
 Missing replicas:		0 (0.0 %)
 Blocks queued for replication:	0

Erasure Coded Block Groups:
 Total size:	0 B
 Total files:	0
 Total block groups (validated):	0
 Minimally erasure-coded block groups:	0
 Over-erasure-coded block groups:	0
 Under-erasure-coded block groups:	0
 Unsatisfactory placement block groups:	0
 Average block group size:	0.0
 M

# WebHDFS

In [33]:
# https://hadoop.apache.org/docs/r3.3.6/hadoop-project-dist/hadoop-hdfs/WebHDFS.html

In [34]:
# Example 1: List a directory: https://hadoop.apache.org/docs/r3.3.6/hadoop-project-dist/hadoop-hdfs/WebHDFS.html#List_a_Directory
# NOTE: we changed ports (using 9870 for WebHDFS/REST APIs, not 9000 like we do for regular command line)
# It looks like "hdfs://" is replaced with "webhdfs/v1"

! curl -i  "http://main:9870/webhdfs/v1/data?op=LISTSTATUS"

HTTP/1.1 200 OK
Date: Fri, 20 Oct 2023 06:52:11 GMT
Cache-Control: no-cache
Expires: Fri, 20 Oct 2023 06:52:14 GMT
Date: Fri, 20 Oct 2023 06:52:14 GMT
Pragma: no-cache
X-Content-Type-Options: nosniff
X-FRAME-OPTIONS: SAMEORIGIN
X-XSS-Protection: 1; mode=block
Content-Type: application/json
Transfer-Encoding: chunked

{"FileStatuses":{"FileStatus":[
{"accessTime":1697765385136,"blockSize":134217728,"childrenNum":0,"fileId":16387,"group":"supergroup","length":15217,"modificationTime":1697765386031,"owner":"root","pathSuffix":"LICENSE.txt","permission":"644","replication":3,"storagePolicy":0,"type":"FILE"},
{"accessTime":1697766068128,"blockSize":134217728,"childrenNum":0,"fileId":16388,"group":"supergroup","length":15217,"modificationTime":1697766068361,"owner":"root","pathSuffix":"v2.txt","permission":"644","replication":1,"storagePolicy":0,"type":"FILE"}
]}}


In [35]:
# -i flag in curl gives the headers, if we remove it we get the plain JSON
! curl "http://main:9870/webhdfs/v1/data?op=LISTSTATUS"

{"FileStatuses":{"FileStatus":[
{"accessTime":1697765385136,"blockSize":134217728,"childrenNum":0,"fileId":16387,"group":"supergroup","length":15217,"modificationTime":1697765386031,"owner":"root","pathSuffix":"LICENSE.txt","permission":"644","replication":3,"storagePolicy":0,"type":"FILE"},
{"accessTime":1697766068128,"blockSize":134217728,"childrenNum":0,"fileId":16388,"group":"supergroup","length":15217,"modificationTime":1697766068361,"owner":"root","pathSuffix":"v2.txt","permission":"644","replication":1,"storagePolicy":0,"type":"FILE"}
]}}


In [39]:
# Example 2: Access a file: https://hadoop.apache.org/docs/r3.3.6/hadoop-project-dist/hadoop-hdfs/WebHDFS.html#Open_and_Read_a_File
# [] means optional
# Read the first 100 bytes of the file
! curl -i -L "http://main:9870/webhdfs/v1/data/v2.txt?op=OPEN&offset=0&length=200"

HTTP/1.1 307 Temporary Redirect
Date: Fri, 20 Oct 2023 06:57:54 GMT
Cache-Control: no-cache
Expires: Fri, 20 Oct 2023 06:57:54 GMT
Date: Fri, 20 Oct 2023 06:57:54 GMT
Pragma: no-cache
X-Content-Type-Options: nosniff
X-FRAME-OPTIONS: SAMEORIGIN
X-XSS-Protection: 1; mode=block
Location: http://main:9864/webhdfs/v1/data/v2.txt?op=OPEN&namenoderpcaddress=main:9000&length=200&offset=0
Content-Type: application/octet-stream
Content-Length: 0

HTTP/1.1 200 OK
Access-Control-Allow-Methods: GET
Access-Control-Allow-Origin: *
Content-Type: application/octet-stream
Connection: close
Content-Length: 200


                                 Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUC

In [40]:
# Notice how if we keep the length and change the offset, the printout changes (by 1 letter)
! curl -i -L "http://main:9870/webhdfs/v1/data/v2.txt?op=OPEN&offset=1&length=200"

HTTP/1.1 307 Temporary Redirect
Date: Fri, 20 Oct 2023 06:58:06 GMT
Cache-Control: no-cache
Expires: Fri, 20 Oct 2023 06:58:06 GMT
Date: Fri, 20 Oct 2023 06:58:06 GMT
Pragma: no-cache
X-Content-Type-Options: nosniff
X-FRAME-OPTIONS: SAMEORIGIN
X-XSS-Protection: 1; mode=block
Location: http://main:9864/webhdfs/v1/data/v2.txt?op=OPEN&namenoderpcaddress=main:9000&length=200&offset=1
Content-Type: application/octet-stream
Content-Length: 0

HTTP/1.1 200 OK
Access-Control-Allow-Methods: GET
Access-Control-Allow-Origin: *
Content-Type: application/octet-stream
Connection: close
Content-Length: 200

                                 Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCT

In [41]:
# curl -L means "follow redirect". When we send a request to the name node, it uses HTTP redirect to direct our 
# request to an appropriate data node
# For example, if we don't use -L, we don't get any data back (the headers are because of the -i flag)
! curl -i "http://main:9870/webhdfs/v1/data/v2.txt?op=OPEN&offset=1&length=200"

HTTP/1.1 307 Temporary Redirect
Date: Fri, 20 Oct 2023 07:00:55 GMT
Cache-Control: no-cache
Expires: Fri, 20 Oct 2023 07:00:55 GMT
Date: Fri, 20 Oct 2023 07:00:55 GMT
Pragma: no-cache
X-Content-Type-Options: nosniff
X-FRAME-OPTIONS: SAMEORIGIN
X-XSS-Protection: 1; mode=block
Location: http://main:9864/webhdfs/v1/data/v2.txt?op=OPEN&namenoderpcaddress=main:9000&length=200&offset=1
Content-Type: application/octet-stream
Content-Length: 0



In [42]:
# Using the noredirect in the URL just gives us the location of the blocks we're trying to access (using the offset and length)
! curl -i "http://main:9870/webhdfs/v1/data/v2.txt?op=OPEN&offset=1&length=200&noredirect=true"

HTTP/1.1 200 OK
Date: Fri, 20 Oct 2023 07:03:25 GMT
Cache-Control: no-cache
Expires: Fri, 20 Oct 2023 07:03:25 GMT
Date: Fri, 20 Oct 2023 07:03:25 GMT
Pragma: no-cache
X-Content-Type-Options: nosniff
X-FRAME-OPTIONS: SAMEORIGIN
X-XSS-Protection: 1; mode=block
Content-Type: application/json
Transfer-Encoding: chunked

{"Location":"http://main:9864/webhdfs/v1/data/v2.txt?op=OPEN&namenoderpcaddress=main:9000&length=200&offset=1"}

# Using Python requests module instead of curl

In [43]:
import requests

In [48]:
r = requests.get("http://main:9870/webhdfs/v1/data/v2.txt?op=OPEN&offset=1&length=200&noredirect=true")
r.raise_for_status()  # raise an exception if we didn't get a return code of 200 (success)
# r.json()  # convert the request response from bytes to JSON
r.json()["Location"]  # now that the response is in JSON, use standard JSON syntax to access an value by key

'http://main:9864/webhdfs/v1/data/v2.txt?op=OPEN&namenoderpcaddress=main:9000&length=200&offset=1'

In [51]:
r = requests.get("http://main:9870/webhdfs/v1/data/v2.txt?op=OPEN&offset=1&length=200")
r.raise_for_status()  
r.content  # this response doesn't give JSON (we omitted the noredirect=true portion of the query) so using the .json() method won't work here

b'                                 Apache License\n                           Version 2.0, January 2004\n                        http://www.apache.org/licenses/\n\n   TERMS AND CONDITIONS FOR USE, REPRODUCT'

# PyArrow - for easier working with files

In [52]:
import pyarrow as pa
import pyarrow.fs

In [53]:
# pa.fs.HadoopFileSystem(<HOST>, <PORT>)
# also note we're not using WebHDFS here, so just use port 9000 again (not 9870)
hdfs = pa.fs.HadoopFileSystem("main", 9000)

2023-10-20 07:15:35,134 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [74]:
f = hdfs.open_input_file("/data/v2.txt")

In [56]:
type(f)

pyarrow.lib.NativeFile

In [57]:
type(f).__mro__  # method resolution order, can tell us about method inheritance, if applicable

(pyarrow.lib.NativeFile, pyarrow.lib._Weakrefable, object)

In [60]:
dir(f)  # tells us the methods and attributes available with this f object
# reviewing the methods and attributes tells us that this is similar to a Python raw file (no text, just bytes)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_assert_open',
 '_assert_readable',
 '_assert_seekable',
 '_assert_writable',
 '_default_chunk_size',
 'close',
 'closed',
 'download',
 'fileno',
 'flush',
 'get_stream',
 'isatty',
 'metadata',
 'mode',
 'read',
 'read1',
 'read_at',
 'read_buffer',
 'readable',
 'readall',
 'readinto',
 'readline',
 'readlines',
 'seek',
 'seekable',
 'size',
 'tell',
 'truncate',
 'upload',
 'writable',
 'write',
 'writelines']

In [61]:
f.read_at(500, 0)  # we can see the 500 bytes starting at offset 0

b'\n                                 Apache License\n                           Version 2.0, January 2004\n                        http://www.apache.org/licenses/\n\n   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION\n\n   1. Definitions.\n\n      "License" shall mean the terms and conditions for use, reproduction,\n      and distribution as defined by Sections 1 through 9 of this document.\n\n      "Licensor" shall mean the copyright owner or entity authorized by\n      the copyright owner that i'

In [63]:
# but we can't loop over it in this format --> notice how this gives an error
for line in f:
    print(line)

UnsupportedOperation: 

In [64]:
import io

In [65]:
reader = io.BufferedReader(f)
for line in reader:
    print(line)  # line is still in bytes if we do this
    break

b'\n'


In [78]:
# now we can loop over the file as if it's text
with hdfs.open_input_file("/data/v2.txt") as f:
    reader = io.TextIOWrapper(io.BufferedReader(f))
    for i, line in enumerate(reader):
        print(line)
        if i > 10: break



                                 Apache License

                           Version 2.0, January 2004

                        http://www.apache.org/licenses/



   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION



   1. Definitions.



      "License" shall mean the terms and conditions for use, reproduction,

      and distribution as defined by Sections 1 through 9 of this document.



